In [22]:
import pandas as pd
import json

In [24]:
dbpedia_df = pd.read_csv("../datasets/dbpedia.csv", sep="|")

In [27]:
faketest_ids = dbpedia_df.sample(4400, random_state=42).id.values # Test set from organizers has ca. 4400 examples

In [29]:
dbpedia_json = None

with open('../datasets/DBpedia/smarttask_dbpedia_train.json') as json_file:
    dbpedia_json = json.load(json_file)

In [32]:
faketest = list()
added_ids = list()

for q in dbpedia_json:
    if q['id'] in faketest_ids and q['id'] not in added_ids:
        faketest.append(q)
        added_ids.append(q['id'])

In [33]:
len(faketest)

4380

In [35]:
with open('../datasets/DBpedia/faketest_dbpedia.json', 'w') as json_file:
    json.dump(faketest, json_file)

In [34]:
"dbpedia_8947" in faketest_ids

True

In [20]:
faketest_preds_json = None

with open('../evaluation/faketest_dbpedia_pred.json') as json_file:
    faketest_preds_json = json.load(json_file)

In [21]:
len(faketest_preds_json)

4400